In [1]:
from modelzipper.tutils import *
from transformers import AutoTokenizer
from datasets import Dataset, DatasetDict
import random

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
dir_path = "/mnt/petrelfs/tangzecheng/local_data/processed_multi_hop/random_drop/llama"
file_names = auto_read_dir(dir_path)
file_names.sort()
content_drop_1 = auto_read_data(os.path.join(dir_path, file_names[0]))
content_drop_2 = auto_read_data(os.path.join(dir_path, file_names[1]))
content_drop_3 = auto_read_data(os.path.join(dir_path, file_names[2]))

/mnt/petrelfs/tangzecheng/anaconda3/envs/zecheng_new/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModelZipper is ready for launch🚀 | Current Version🦄 >>> 0.2.7 <<< | AOE Time🕒 2024-12-25 16:36:22


2024-12-25 12:36:44.670 | INFO     | modelzipper.tutils:auto_read_dir:369 - number of files with prefix '' and suffix '': 3


begin to read data from /mnt/petrelfs/tangzecheng/local_data/processed_multi_hop/random_drop/llama/inference_drop_1.pkl | file size: 1.25 GB | file type: pkl
begin to read data from /mnt/petrelfs/tangzecheng/local_data/processed_multi_hop/random_drop/llama/inference_drop_2.pkl | file size: 1.22 GB | file type: pkl
begin to read data from /mnt/petrelfs/tangzecheng/local_data/processed_multi_hop/random_drop/llama/inference_drop_3.pkl | file size: 1.2 GB | file type: pkl


### 下面的数据集仅用来进行测试开发使用，一共32条训练，32条测试

In [4]:
# 首先从每个数据集中取相同数目的数据出来
all_training_data = []

all_content_data = content_drop_1 + content_drop_2 + content_drop_3
all_content_data = all_content_data[:64]
for item in all_content_data:
    all_training_data.append({
        "prompt": item["prompt"],
        "chosen": [
            {"role": "user", "content": item["prompt"]}, 
            {"role": "assistant", "content": item["answer"]}
        ],
        "rejected": [
            {"role": "user", "content": item["prompt"]}, 
            {"role": "assistant", "content": item["pred"][0]}
        ],
    })

dataset = Dataset.from_list(all_training_data)

# 从中随机抽取 32 条作为 validation 数据
validation_size = 32

# 打乱数据集索引并取前 32 条作为 validation
indices = list(range(len(dataset)))
random.shuffle(indices)

validation_indices = indices[:validation_size]
train_indices = indices[validation_size:]

# 使用 Hugging Face 的 select 方法创建新的训练集和验证集
train_dataset = dataset.select(train_indices)
validation_dataset = dataset.select(validation_indices)

# 将 train 和 validation 数据集保存到一个 dict 中
dataset_dict = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset
})

# 打印数据集长度验证
print(f"Train dataset size: {len(dataset_dict['train'])}")
print(f"Validation dataset size: {len(dataset_dict['validation'])}")

# 保存到本地
dataset_dict.save_to_disk("/mnt/petrelfs/tangzecheng/local_data/processed_multi_hop/random_drop/train_data/merge_v1_dev")

Train dataset size: 32
Validation dataset size: 32


Saving the dataset (1/1 shards): 100%|██████████| 32/32 [00:00<00:00, 473.08 examples/s]


In [3]:
# 首先从每个数据集中取相同数目的数据出来
all_training_data = []

all_content_data = content_drop_1 + content_drop_2 + content_drop_3

for item in all_content_data:
    all_training_data.append({
        "prompt": item["prompt"],
        "chosen": [
            {"role": "user", "content": item["prompt"]}, 
            {"role": "assistant", "content": item["answer"]}
        ],
        "rejected": [
            {"role": "user", "content": item["prompt"]}, 
            {"role": "assistant", "content": item["pred"][0]}
        ],
    })

dataset = Dataset.from_list(all_training_data)

# 从中随机抽取 500 条作为 validation 数据
validation_size = 500

# 打乱数据集索引并取前 500 条作为 validation
indices = list(range(len(dataset)))
random.shuffle(indices)

validation_indices = indices[:validation_size]
train_indices = indices[validation_size:]

# 使用 Hugging Face 的 select 方法创建新的训练集和验证集
train_dataset = dataset.select(train_indices)
validation_dataset = dataset.select(validation_indices)

# 将 train 和 validation 数据集保存到一个 dict 中
dataset_dict = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset
})

# 打印数据集长度验证
print(f"Train dataset size: {len(dataset_dict['train'])}")
print(f"Validation dataset size: {len(dataset_dict['validation'])}")

# 保存到本地
dataset_dict.save_to_disk("/mnt/petrelfs/tangzecheng/local_data/processed_multi_hop/random_drop/train_data/merge_v1")

Train dataset size: 20500
Validation dataset size: 500


Saving the dataset (1/1 shards): 100%|██████████| 500/500 [00:00<00:00, 585.30 examples/s]


In [3]:
content_drop_1[1]['answer'] == content_drop_2[1]['answer'] == content_drop_3[1]['answer']

True

In [4]:
content_drop_1[0]

{'prompt': 'Answer the question based on the given passages.\n\nThe following are given passages.\nA team led by Alan Hemming, MD, transplant surgeon at UC San Diego Health System, has successfully performed the west coast\'s first ex-vivo liver resection, a radical procedure to completely remove and reconstruct a diseased liver and re-implant it without any tumors. The procedure saved the life of a 27-year old mother whose liver had been invaded by a painful tumor that crushed the organ and entangled its blood supply.\n"During a 9-hour surgery the team was able to remove the basketball-sized tumor," said Hemming, professor and surgical director of the Center for Hepatobiliary Disease and Abdominal Transplantation (CHAT) at UC San Diego Health System. "This is a surgery that carries a 15 to 20 percent risk of mortality. In this case, the patient would not have survived if she did not have surgery. This was the only way we could save her liver and her life."\nDuring the procedure, the d